# Instructor Do: Tone Alysis

In [8]:
# Initial imports
import os
import json
from pandas.io.json import json_normalize
from ibm_watson import ToneAnalyzerV3
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from dotenv import load_dotenv
load_dotenv()
from path import Path

# pip install --upgrade "ibm-watson>=3.0.3"

In [9]:
# Reading the News API key enviroment variable
abs_path = Path(r'C:/Users/metin/Documents/nufintech/.env')
load_dotenv(abs_path) 

tone_api = os.getenv("tone_key")
tone_url = os.getenv("tone_url")

In [10]:
# Initialize Tone Analyser Client

# Create authentication object
authenticator = IAMAuthenticator(tone_api)

# Create tone_analyzer instance
tone_analyzer = ToneAnalyzerV3(
    version="2017-09-21",
    authenticator=authenticator
)

# Set the service endpoint
tone_analyzer.set_service_url(tone_url)


## General Tone Analysis Test

In [11]:
# Define text to analyze
text = """
Team, I know that times are tough!
Product sales have been disappointing for the past three quarters.
We have a competitive product, but we need to do a better job of selling it!
"""

# Analyze the text's tone with the 'tone()' method.
tone_analysis = tone_analyzer.tone(
    {"text": text},
    content_type="application/json",
    content_language="en",
    accept_language="en",
).get_result()

# Display tone analysis results
print(json.dumps(tone_analysis, indent=2))


ApiException: Error: Provided API key could not be found, Code: 400

### Transform Tone Analysis json to a DataFrame

In [22]:
# Document Tones
doc_tone_df = json_normalize(data=tone_analysis["document_tone"], record_path=["tones"])
doc_tone_df



,score,tone_id,tone_name
0,0.616500,sadness,Sadness
1,0.829888,analytical,Analytical


In [26]:
# Sentences Tones
sentences_tone_df = json_normalize(
    data=tone_analysis["sentences_tone"],
    record_path=["tones"],
    meta=["sentence_id", "text"],
)
sentences_tone_df

,score,tone_id,tone_name,sentence_id,text
0,0.801827,analytical,Analytical,0,"Team, I know that times are tough!"
1,0.771241,sadness,Sadness,1,Product sales have been disappointing for the ...
2,0.687768,analytical,Analytical,1,Product sales have been disappointing for the ...
3,0.506763,analytical,Analytical,2,"We have a competitive product, but we need to ..."


## Customer Engagement Tone Analysis

In [27]:
# Define conversational utterances
utterances = [
    {"text": "Hello, I'm having a problem with your product.", "user": "customer"},
    {"text": "OK, let me know what's going on, please.", "user": "agent"},
    {"text": "Well, nothing is working :(", "user": "customer"},
    {"text": "Sorry to hear that.", "user": "agent"},
]

# Analyze utterances using the 'tone_chat()' method
utterance_analysis = tone_analyzer.tone_chat(
    utterances=utterances, content_language="en", accept_language="en"
).get_result()
print(json.dumps(utterance_analysis, indent=2))


{
  "utterances_tone": [
    {
      "utterance_id": 0,
      "utterance_text": "Hello, I'm having a problem with your product.",
      "tones": [
        {
          "score": 0.686361,
          "tone_id": "polite",
          "tone_name": "Polite"
        }
      ]
    },
    {
      "utterance_id": 1,
      "utterance_text": "OK, let me know what's going on, please.",
      "tones": [
        {
          "score": 0.92724,
          "tone_id": "polite",
          "tone_name": "Polite"
        }
      ]
    },
    {
      "utterance_id": 2,
      "utterance_text": "Well, nothing is working :(",
      "tones": [
        {
          "score": 0.997795,
          "tone_id": "sad",
          "tone_name": "Sad"
        }
      ]
    },
    {
      "utterance_id": 3,
      "utterance_text": "Sorry to hear that.",
      "tones": [
        {
          "score": 0.730982,
          "tone_id": "polite",
          "tone_name": "Polite"
        },
        {
          "score": 0.672499,
          "to

### Transform Chat Analysis json to DataFrame

In [28]:
chat_tone_df = json_normalize(
    data=utterance_analysis["utterances_tone"],
    record_path=["tones"],
    meta=["utterance_id", "utterance_text"],
)
chat_tone_df


,score,tone_id,tone_name,utterance_id,utterance_text
0,0.686361,polite,Polite,0,"Hello, I'm having a problem with your product."
1,0.927240,polite,Polite,1,"OK, let me know what's going on, please."
2,0.997795,sad,Sad,2,"Well, nothing is working :("
3,0.730982,polite,Polite,3,Sorry to hear that.
4,0.672499,sympathetic,Sympathetic,3,Sorry to hear that.
